In [ ]:
# --- Cell 1: Imports and DLL setup ---
import numpy as np
from ctypes import *
import sys
sys.path.append("c:/Users/mainUED/Documents/GitHub/UED/GUI/")

# Adjust path to your DLL
#dll_path = r".\DLLs\GladzDLL.dll"  
glaz = cdll.LoadLibrary("./DLLs/gladz/GlazLib.dll")


initial_path = r"."  # or wherever your scripts are located


In [ ]:
# --- Cell 2: Initialize Gladz ---
def initialize_Gladz():
    global glaz
    script_path = initial_path + r'\DLLs\trial.gsc'
    
    # encode script path to bytes
    script = c_char_p(bytes(script_path, 'utf-8'))
    
    res = glaz.initialiseSession(script)
    print("Initialise result:", res)

    error_message = c_char_p(b"No error detected")
    res = glaz.getLastErrorMessage(error_message)
    print("Last error message: ", error_message.value)

initialize_Gladz()


In [ ]:
# --- Cell 3: Set scan parameters ---
scan_cnt = 100   # manual replacement for lineEdit_4.text()
scan_cnt_out = c_int(0)
pd_number = c_int(1)   # must match your XML config
pd_channel = c_int(2)  # channel selection

def set_GladzPD():
    global scan_cnt, glaz
    res = glaz.setScanCount(c_int(scan_cnt))
    print("setScanCount result:", res)
    
    if res != 0:
        error_message = c_char_p(b"No error detected")
        res = glaz.getLastErrorMessage(error_message)
        print("Error:", error_message.value)

set_GladzPD()


In [ ]:
# --- Cell 4: Run measurement and get data ---
def Gladz_data():
    # Step 1: run a measurement
    res = glaz.runMeasurement()
    print("runMeasurement result:", res)

    # Step 2: fetch values
    data = np.ndarray(scan_cnt, dtype=np.double)
    data_ctype = data.ctypes.data_as(POINTER(c_double))

    glaz.getPDValues.argtypes = [c_int, c_int, POINTER(c_int), POINTER(c_double)]
    _ = glaz.getPDValues(pd_number, pd_channel, byref(scan_cnt_out), data_ctype)

    error_message = c_char_p(b"No error detected")
    _ = glaz.getLastErrorMessage(error_message)
    if error_message.value != b"":
        print("Error:", error_message.value)

    return data, scan_cnt_out.value

data_diode, scan_out = Gladz_data()
print("Retrieved", scan_out, "samples")
print(data_diode)


In [ ]:
# --- Cell 5: Simple plotting ---
import matplotlib.pyplot as plt

def gladz_plot(data):
    plt.figure(figsize=(8,4))
    plt.plot(data)
    plt.title("Gladz Photodiode Data")
    plt.xlabel("Index")
    plt.ylabel("Signal")
    plt.show()

gladz_plot(data_diode)
